In [1]:
import stim
from dem_builder_teleportation_LDU import DEM_builder
import os
import numpy as np
import scipy
from surface_code_teleportation_LDU import Rotated_Surface_Code_teleportation_LDU
import pymatching
import pickle
import multiprocessing
from multiprocessing import Pool

In [2]:
#function for simulation

#Compute the logical error rate of the surface code supplemented with the teleportation LDU for a given noise and loss probability
#stored as a dictionnary in a file for a fixed loss probability and various depolarizing noise probabilities
def simulation_code_fixed_loss(L,loss,depo,num_shots):
    loss=round(loss,5)
    depo=round(depo,5)
    lattice = Rotated_Surface_Code_teleportation_LDU(L,L,L,'|0>','Z',measurement_order='z_first',loss_rate=loss,after_CZ=('DEPOLARIZE2',depo))
    dem_build=DEM_builder(lattice)
    PATH='data/dem/dem_teleportation_LDU/DEM_surface_code_({},{},{})'.format(L,L,L)
    
    error_rate=dem_build.logical_errors(num_shots,PATH)

    
    if os.path.exists('data/logical_error/logical_error_teleportation_LDU_d=({},{})_rounds={}_loss_rate={}.pkl'.format(L,L,L,loss)):
        with open('data/logical_error/logical_error_teleportation_LDU_d=({},{})_rounds={}_loss_rate={}.pkl'.format(L,L,L,loss), 'rb') as fp:
            data_saved=pickle.load(fp)
        
    else:
        data_saved={}
    if depo not in data_saved.keys():#CREATE DICTIONNARY if the noise probability has never been computed
        data_saved[depo]={}
        data_saved[depo]['num_shots']=num_shots
        data_saved[depo]['error_rate']=error_rate
    else: #UPDATE DICTIONNARY if there has been already simulation for that given loss and noise probability
        data_saved[depo]['error_rate']*=data_saved[depo]['num_shots']/(data_saved[depo]['num_shots']+num_shots)
        data_saved[depo]['error_rate']+=error_rate*num_shots/(data_saved[depo]['num_shots']+num_shots)
        data_saved[depo]['num_shots']+=num_shots

    #stored in a file for a given loss and various noise probabilities
    with open('data/logical_error/logical_error_teleportation_LDU_d=({},{})_rounds={}_loss_rate={}.pkl'.format(L,L,L,loss), 'wb') as fp:
        pickle.dump(data_saved, fp)

    print('logical error rate for (L={},loss_rate={},depo={}) = {}'.format(L,loss,depo,error_rate))
    

#wrapper to one argument (needed for parallelization)
def multi_args_wrapper_fixed_loss(args):
   simulation_code_fixed_loss(*args)
    
        

In [3]:
#Compute the logical error rate and store as a dictionnary in files with a fixed loss probabilities

Ls = range(3,13,2)
depo_list = np.linspace(0.001, 0.016, 16) #list of depolarizing noise probabilities
loss_list=[0.,0.001,0.002,0.003,0.005,0.007,0.01,0.015,0.02,0.025,0.027] #list of loss probabilities
num_shots = 10000 #number of shots per core
nb_core=10 #number of cores used

for loss_rate in loss_list:
    for L in Ls:
        for depo in depo_list:
            with Pool(processes=nb_core) as pool:
                pool.imap(multi_args_wrapper_fixed_loss,[(L,loss_rate,depo,num_shots) for _ in range(10)])
                pool.close()
                pool.join()
                

logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0017
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0019
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0019
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0017
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0016
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0013
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0014
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0017
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0015
logical error rate for (L=3,loss_rate=0.002,depo=0.001) = 0.0016
logical error rate for (L=3,loss_rate=0.002,depo=0.002) = 0.0037
logical error rate for (L=3,loss_rate=0.002,depo=0.002) = 0.0037
logical error rate for (L=3,loss_rate=0.002,depo=0.002) = 0.0037
logical error rate for (L=3,loss_rate=0.002,depo=0.002) = 0.0034
logical error rate for (L=3,loss_rate=0.002,depo=0.002) = 0.0042
logical error rate for (L

In [3]:
#function for simulation

#Compute the logical error rate of the surface code supplemented with the teleportation LDU for a given noise and loss probability
#stored as a dictionnary in a file for a fixed depolarizing noise probability and various loss probabilities
def simulation_code_fixed_depo(L,loss,depo,num_shots):
    loss=round(loss,5)
    depo=round(depo,5)
    lattice = Rotated_Surface_Code_teleportation_LDU(L,L,L,'|0>','Z',measurement_order='z_first',loss_rate=loss,after_CZ=('DEPOLARIZE2',depo))
    dem_build=DEM_builder(lattice)
    PATH='data/dem/dem_teleportation_LDU/DEM_surface_code_({},{},{})'.format(L,L,L)
    
    error_rate=dem_build.logical_errors(num_shots,PATH)
    
    if os.path.exists('data/logical_error/logical_error_teleportation_LDU_d=({},{})_rounds={}_depo_noise={}.pkl'.format(L,L,L,depo)):
        with open('data/logical_error/logical_error_teleportation_LDU_d=({},{})_rounds={}_depo_noise={}.pkl'.format(L,L,L,depo), 'rb') as fp:
            data_saved=pickle.load(fp)
        
    else:
        data_saved={}
    if loss not in data_saved.keys():#CREATE DICTIONNARY if the loss probability has never been computed
        data_saved[loss]={}
        data_saved[loss]['num_shots']=num_shots
        data_saved[loss]['error_rate']=error_rate
    else:#UPDATE DICTIONNARY if there has been already simulation for that given loss and noise probability
        data_saved[loss]['error_rate']*=data_saved[loss]['num_shots']/(data_saved[loss]['num_shots']+num_shots)
        data_saved[loss]['error_rate']+=error_rate*num_shots/(data_saved[loss]['num_shots']+num_shots)
        data_saved[loss]['num_shots']+=num_shots

    #stored in a file for a given loss and various noise probabilities
    with open('data/logical_error/logical_error_teleportation_LDU_d=({},{})_rounds={}_depo_noise={}.pkl'.format(L,L,L,depo), 'wb') as fp:
        pickle.dump(data_saved, fp)

    print('logical error rate for (L={},loss_rate={},depo={}) = {}'.format(L,loss,depo,error_rate))
    

#wrapper to one argument (needed for parallelization)    
def multi_args_wrapper_fixed_depo(args):
   simulation_code_fixed_depo(*args)
    
        

In [5]:
#Compute the logical error rate and store as a dictionnary in files with a fixed depolarizing noise probabilities

Ls = range(3,13,2)
loss_list = np.linspace(0.001, 0.03, 30)
depo=0
num_shots = 10000 #number of shots per core
nb_core=10  #number of cores used

for L in Ls:
    for loss_rate in loss_list:
        with Pool(processes=nb_core) as pool:
            pool.imap(multi_args_wrapper_fixed_depo,[(L,loss_rate,depo,num_shots) for _ in range(10)])
            pool.close()
            pool.join()
            

logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0001
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.001,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.002,depo=0.0) = 0.0001
logical error rate for (L=3,loss_rate=0.002,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.002,depo=0.0) = 0.0002
logical error rate for (L=3,loss_rate=0.002,depo=0.0) = 0.0001
logical error rate for (L=3,loss_rate=0.002,depo=0.0) = 0.0
logical error rate for (L=3,loss_rate=0.002,depo=0.0) = 0.0
logical error rate for (L=3,

KeyboardInterrupt: 